In [ ]:
# Install Libraries
# !pip install textblob
# !pip install tweepy
# !pip install pycountry
# !pip3 install nltk 
# !pip3 install -U setuptools wheel
# !pip3 install spacy 
# !pip3 install pandas numpy seaborn matplotlib
# !python3 -m spacy download en_core_web_sm
# !pip3 install gensim
# !pip install wordcloud
# !pip install langdetect
# !pip install pyquery
# !pip install lxml
# nltk.download('vader_lexicon')  
# !pip install requests
# !pip install schedule

In [3]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import requests
import json
import schedule
import time
import datetime
import csv
from wordcloud import WordCloud, STOPWORDSf
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer


In [12]:
# Authentication
consumer_Key = 'chie8F9xbOWpMDOxJTolC89yy'
consumer_Secret = 'WUQwRqXd7niAkOEBzZ5iOH76c4NCS6UunTNJ4IUSFuWempLcBQ'
access_Token = '1356292685385105408-yP2gaDQkkHMSSa2hUcjfwSKueKen8v'
access_Token_Secret = 'UlBapYsreywE4CyYARnEJmqDXlV5KrhXhgvWMmQGIoq3E'

auth = tweepy.OAuthHandler(consumer_Key, consumer_Secret)
auth.set_access_token(access_Token, access_Token_Secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

# curl -u 'chie8F9xbOWpMDOxJTolC89yy:WUQwRqXd7niAkOEBzZ5iOH76c4NCS6UunTNJ4IUSFuWempLcBQ' \
#   --data 'grant_type=client_credentials' \
#   'https://api.twitter.com/oauth2/token'

In [15]:
#its bad practice to place your bearer token directly into the script (this is just done for illustration purposes)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPgDNwEAAAAA5LZB%2F%2FVS6CkUQKsWmVwBVyQKXHk%3DUSoT6IfI2Fi052WlIqZjbOM1Duw1JTdm74ofZw6xvUwI1Z28sW"
#define search twitter function
def search_twitter(query, tweet_fields, expansions, place_fields, bearer_token = BEARER_TOKEN):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&{}&{}".format(
        query, tweet_fields, expansions, place_fields
    )
    response = requests.request("GET", url, headers=headers)

    print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [6]:
#search term
query = "(online school) OR (online learning) OR (distance learning) OR (online classes) OR (online class) OR (remote learning) -filter:retweets"
#twitter fields to be returned by api call
tweet_fields = "tweet.fields=text,author_id,created_at,geo"
expansions = "expansions=geo.place_id"
place_fields = "place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type"

#twitter api call
json_response = search_twitter(query=query, tweet_fields=tweet_fields, expansions=expansions, place_fields = place_fields, bearer_token=BEARER_TOKEN)
#pretty printing
print(json.dumps(json_response, indent=4, sort_keys=True))

200
{
    "data": [
        {
            "author_id": "134391739",
            "created_at": "2021-03-21T04:44:55.000Z",
            "id": "1373495817424629764",
            "text": "@PLDTHome @PLDT_Cares 3 to 4 days??? Without prior notice??? So paano yung mga students with online classes and work from home??? \ud83d\ude24\ud83e\udd2c worse, it takes your reps hours to respond to a simple inquiry! @iamMVP https://t.co/izxYY4hkqQ"
        },
        {
            "author_id": "1186700905296617478",
            "created_at": "2021-03-21T04:44:46.000Z",
            "id": "1373495776517718017",
            "text": "RT @priinnyvert: online school sucks because i can\u2019t ask the person sitting next to me what we\u2019re supposed to be doing"
        },
        {
            "author_id": "850836534",
            "created_at": "2021-03-21T04:44:43.000Z",
            "id": "1373495763964026893",
            "text": "RT @cjmorphosis: KAPAG GRUMADUATE NA KAMING UNANG BATCH NG K TO 12  WAG KA

In [33]:
# Define the search term and the date_since date as variables
search_words = "(online school) OR (online learning) OR (distance learning) OR (online classes) OR (online class) OR (remote learning) OR (blended learning) OR (elearning)"
date_since = "2021-02-01"
new_search = search_words + " -filter:retweets"
noOfTweet = 100000

tweets = tweepy.Cursor(api.search, 
                           q=new_search,
                           geo="-25.208749,-55.270782,10000km",
                           lang="en",
                           since=date_since).items(noOfTweet)

users_locs = [[tweet.user.screen_name, tweet.text, tweet.user.location, tweet.created_at] for tweet in tweets]
# users_locs

KeyboardInterrupt: 

In [32]:
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user', "tweet","location"])
tweet_text

,user,tweet,location
0,Writer_Moha,Pay for:-\n*Assignments\n*Maths\n*Homeworks\n*...,United States
1,monkey_viral,#Important Elements Of eLearning Courses is no...,Globe & Beyond
2,GStottor,Thank you @fleurmurray and @pineappledance for...,"Brighton, England"
3,Hawksy14,Online classes have effected my mental health ...,
4,zChgxd,@enthusieastro Me too. Online class is too exh...,STAN ASTRO
...,...,...,...
2356,SamAbhi10,@DrRPNishank sir.our parents are not sending f...,
2357,spn__weirdo,@P3tty0dd Oh yeah I tried to get that but my m...,18: any pronouns:♌️
2358,ELMIT,Did you lose your job? Are you interested in l...,Taipei
2359,SamAbhi10,@DrRPNishank sir.our parents are not sending f...,


In [29]:
print(tweet_text[tweet_text['location'].str.contains("United Arab Emirates")])

              user                                              tweet  \
1240         Meirc  Certificate in Professional Arabic Writing Ski...   
1688   MrAndiPrice  Just love using @PhETsims FREE (Yes FREE!👏) ℂ𝕚...   
1978  MicrosoftUAE  Technology played an important role in adaptin...   

                         location  
1240  Dubai, United Arab Emirates  
1688  Dubai, United Arab Emirates  
1978         United Arab Emirates  
